In [1]:
assignments = []
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a, b):
    return [s+t for s in a for t in b]

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diagonal_units1 = [[i+j for i,j in zip(rows, cols)]]
diagonal_units2 = [[i+j for i,j in zip(rows,sorted(cols,reverse=True))]]
unitlist = row_units + column_units + square_units + diagonal_units1 + diagonal_units2
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in a for t in b]

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Input: A grid in string form.
    Output: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))
    
def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    #print('before reducing',values)
    values = reduce_puzzle(values)
    #print('after reducing',values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

# def solve(grid):
#     """
#     Find the solution to a Sudoku grid.
#     Args:
#         grid(string): a string representing a sudoku grid.
#             Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
#     Returns:
#         The dictionary representation of the final sudoku grid. False if no solution exists.
#     """

# if __name__ == '__main__':
#     diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
#     display(solve(diag_sudoku_grid))

#     try:
#         from visualize import visualize_assignments
#         visualize_assignments(assignments)

#     except SystemExit:
#         pass
#     except:
#         print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


In [2]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    #unitlist.append(diagonal_units)
    values = grid_values(grid)
    search(values)
    return values

In [4]:
for unit in unitlist:
    print(unit)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9']
['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9']
['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9']
['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9']
['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9']
['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']
['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2']
['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3']
['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4']
['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5']
['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6']
['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7']
['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8']
['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'I9']
['A1', 'A2

In [14]:
def nt(values):
    
    for unit in unitlist:
        mydict = {}
        for box in unit:
            x = values[box]
            if(len(x)) == 2:
                if x not in mydict.keys():
                    mydict[x] = 1
                else:
                    mydict[x] = mydict.get(x) + 1
        
        mydictlist = [k for k,v in mydict.items() if v == 2]
        for box in unit:
            for item in mydictlist:
                for i in item:
                    if values[box] != item and i in values[box]:
                        values[box] = values[box].replace(i,'')
                        
    return values

In [15]:
values = nt(values)
print(values)

{'A1': '', 'A2': '16', 'A3': '7', 'A4': '9', 'A5': '4', 'A6': '16', 'A7': '3', 'A8': '8', 'A9': '', 'B1': '8', 'B2': '5', 'B3': '3', 'B4': '7', 'B5': '1', 'B6': '6', 'B7': '2', 'B8': '4', 'B9': '9', 'C1': '4', 'C2': '29', 'C3': '29', 'C4': '8', 'C5': '168', 'C6': '3', 'C7': '15', 'C8': '7', 'C9': '6', 'D1': '5', 'D2': '7', 'D3': '6', 'D4': '4', 'D5': '3', 'D6': '8', 'D7': '1', 'D8': '9', 'D9': '2', 'E1': '3', 'E2': '8', 'E3': '4', 'E4': '1', 'E5': '9', 'E6': '2', 'E7': '6', 'E8': '5', 'E9': '7', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6', 'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'G1': '6', 'G2': '4', 'G3': '2', 'G4': '3', 'G5': '7', 'G6': '9', 'G7': '8', 'G8': '1', 'G9': '5', 'H1': '9', 'H2': '3', 'H3': '5', 'H4': '2', 'H5': '8', 'H6': '1', 'H7': '7', 'H8': '6', 'H9': '4', 'I1': '7', 'I2': '1', 'I3': '8', 'I4': '5', 'I5': '6', 'I6': '4', 'I7': '9', 'I8': '2', 'I9': '3'}


In [13]:
#values='2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
#solve(values)
#values
#display(values)

values={'A1': '2', 'A2': '16', 'A3': '7', 'A4': '9', 'A5': '4', 'A6': '16', 'A7': '3', 'A8': '8', 'A9': '1', 'B1': '8', 'B2': '5', 'B3': '3', 'B4': '7', 'B5': '1', 'B6': '6', 'B7': '2', 'B8': '4', 'B9': '9', 'C1': '24', 'C2': '29', 'C3': '29', 'C4': '8', 'C5': '1268', 'C6': '39', 'C7': '1295', 'C8': '7', 'C9': '6', 'D1': '5', 'D2': '7', 'D3': '6', 'D4': '4', 'D5': '3', 'D6': '8', 'D7': '1', 'D8': '9', 'D9': '2', 'E1': '3', 'E2': '8', 'E3': '4', 'E4': '1', 'E5': '9', 'E6': '2', 'E7': '6', 'E8': '5', 'E9': '7', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6', 'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'G1': '6', 'G2': '4', 'G3': '2', 'G4': '3', 'G5': '7', 'G6': '9', 'G7': '8', 'G8': '1', 'G9': '5', 'H1': '9', 'H2': '3', 'H3': '5', 'H4': '2', 'H5': '8', 'H6': '1', 'H7': '7', 'H8': '6', 'H9': '4', 'I1': '7', 'I2': '1', 'I3': '8', 'I4': '5', 'I5': '6', 'I6': '4', 'I7': '9', 'I8': '2', 'I9': '3'}

In [2]:
%run ./solution_test.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.011s

OK


In [79]:
%tb

SystemExit: True

In [30]:
[[i+j for i,j in zip(rows, cols)] + [i+j for i,j in zip(rows,sorted(cols,reverse=True))]]

[['A1',
  'B2',
  'C3',
  'D4',
  'E5',
  'F6',
  'G7',
  'H8',
  'I9',
  'A9',
  'B8',
  'C7',
  'D6',
  'E5',
  'F4',
  'G3',
  'H2',
  'I1']]

In [8]:
sorted(cols,reverse=True)

['9', '8', '7', '6', '5', '4', '3', '2', '1']

In [92]:
%run ./solution.py

TypeError: 'NoneType' object is not subscriptable